## Assignment-6: Machine Learning on Text

**Reddit Comment Score Prediction**

Reddit is a social news platform that allows users to discuss and vote on content that other users have submitted.On an average reddit receives 470,000 comments per day. The comments are further upvoted or downvoted by the registered users. 

Imagine you are going to start a forum where users can post or comment or share content on the platform. Now you want to filter out some positive comments and recommend them to your users. 


Build a machine learning model that will help you know which comment or content is going to be popular in the near future (the content which receives the highest upvotes will be popular) and accordingly recommend such content to your users.

In [1]:
pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl (300.9 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp311-cp311-win_amd64.whl (2.7 MB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl (938 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached wrapt-1.14.1-cp311-cp311-win_amd64.whl (35 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl (1.5 MB)
  Using cached grpcio-1.62.0-cp311-cp311-win_amd64.whl (3.8 MB)
  Using cached tensorboard-2.15.2-py3-none-

In [3]:
pip install tensorflow_hub


                                              0.0/1.7 MB ? eta -:--:--
     -----                                    0.2/1.7 MB 3.8 MB/s eta 0:00:01
     --------                                 0.4/1.7 MB 2.3 MB/s eta 0:00:01
     -------------                            0.6/1.7 MB 2.3 MB/s eta 0:00:01
     -----------------------                  1.0/1.7 MB 2.8 MB/s eta 0:00:01
     --------------------------------         1.4/1.7 MB 3.2 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     -------------------------------------    1.6/1.7 MB 3.3 MB/s eta 0:00:01
     ---------------------------------------  1.7/1.7 MB 2.2 MB/s eta 


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model, load_model
import nltk
nltk.download('vader_lexicon')
!pip3 install -U nltk[twitter] 
!pip install sentencepiece

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/991.5 kB ? eta -:--:--
     ---                                     81.9/991.5 kB 2.3 MB/s eta 0:00:01
     ---------                              245.8/991.5 kB 3.0 MB/s eta 0:00:01
     ----------------                       440.3/991.5 kB 3.1 MB/s eta 0:00:01
     -------------------------              665.6/991.5 kB 3.5 MB/s eta 0:00:01
     ------------------------------         798.7/991.5 kB 3.6 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  983


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
df_compi = pd.read_csv("Train_Data.csv")
df_compi_test = pd.read_csv("Test_Data.csv")

In [6]:
# mark postive and negative sentiment
df_compi["positive"] = 1
df_compi.loc[(df_compi.Score <0), "positive"] = 0
df_compi.positive.value_counts()

positive
1    2504
0    2495
Name: count, dtype: int64

# Dropping duplicate columns

In [7]:
# check if "parent_votes" and "parent_score" columns contains the same value
(df_compi.parent_votes == df_compi.parent_score).all()

True

In [8]:
# drop duplicate and redundunt columns
red_col_list = ["author", "parent_author"]
df_compi.drop(["parent_votes"] + red_col_list, axis= 1,inplace= True)
df_compi_test.drop(["parent_votes"] + red_col_list, axis= 1,inplace= True)

# Adding VADER polarity scores

In [10]:
analyzer = SentimentIntensityAnalyzer()

def add_polarity_score(df):
    """
    Add sentiment scores
    """
    # text column names
    text_col = ["text", "parent_text"]
    
    # for each text column
    for col in text_col:
        neg = []
        neu = []
        pos = []
        compound = []
        # for each text
        for text in df[col].values.flatten():
            vs = analyzer.polarity_scores(text)
            neg.append(vs["neg"])
            neu.append(vs["neu"])
            pos.append(vs["pos"])
            compound.append(vs["compound"])
    
        # add column to dataframe
        df[f"neg_{col}"] = neg
        df[f"neu_{col}"] = neu
        df[f"pos_{col}"] = pos
        df[f"compound_{col}"] = compound

In [11]:
# add polarity score for each dataframe
add_polarity_score(df_compi)
add_polarity_score(df_compi_test)

# Train validation split on competition data

In [12]:
#### from sklearn.model_selection import train_test_split
# split the data into train and valudation data(10%) with Stratify sampling, random state 33, 
X_train, X_val = train_test_split( df_compi, test_size=0.1, random_state=33)
X_test = df_compi_test

# Creating BERT Model

In [13]:
# Loading the Pretrained Model from tensorflow HUB
tf.keras.backend.clear_session()

# maximum length of a seq in the data we have, for now i am making it as 300. You can change this
max_seq_length = 300

#BERT takes 3 inputs

#this is input words. Sequence of words represented as integers
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")

#mask vector if you are padding anything
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")

#segment vectors. If you are giving only one sentence for the classification, total seg vector is 0. 
#If you are giving two sentenced with [sep] token separated, first seq segment vectors are zeros and 
#second seq segment vector are 1's
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

#bert layer 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

#Bert model
#We are using only pooled output not sequence out. 
#If you want to know about those, please read https://www.kaggle.com/questions-and-answers/86510
bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)

In [14]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 300)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 300)]                0         []                            
                                                                                                  
 segment_ids (InputLayer)    [(None, 300)]                0         []                            
                                                                                                  
 keras_layer (KerasLayer)    [(None, 768),                1094822   ['input_word_ids[0][0]',  

# Tokenization

In [15]:
# getting Vocab file
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [16]:
# coding=utf-8
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Tokenization classes implementation.

The file is forked from:
https://github.com/google-research/bert/blob/master/tokenization.py.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import unicodedata

import six
import tensorflow as tf

import sentencepiece as spm

SPIECE_UNDERLINE = "▁"


def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
  """Checks whether the casing config is consistent with the checkpoint name."""

  # The casing has to be passed in by the user and there is no explicit check
  # as to whether it matches the checkpoint. The casing information probably
  # should have been stored in the bert_config.json file, but it's not, so
  # we have to heuristically detect it to validate.

  if not init_checkpoint:
    return

  m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
  if m is None:
    return

  model_name = m.group(1)

  lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
  ]

  cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
  ]

  is_bad_config = False
  if model_name in lower_models and not do_lower_case:
    is_bad_config = True
    actual_flag = "False"
    case_name = "lowercased"
    opposite_flag = "True"

  if model_name in cased_models and do_lower_case:
    is_bad_config = True
    actual_flag = "True"
    case_name = "cased"
    opposite_flag = "False"

  if is_bad_config:
    raise ValueError(
        "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
        "However, `%s` seems to be a %s model, so you "
        "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
        "how the model was pre-training. If this error is wrong, please "
        "just comment out this check." %
        (actual_flag, init_checkpoint, model_name, case_name, opposite_flag))


def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  with tf.io.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab


def convert_by_vocab(vocab, items):
  """Converts a sequence of [tokens|ids] using the vocab."""
  output = []
  for item in items:
    output.append(vocab[item])
  return output


def convert_tokens_to_ids(vocab, tokens):
  return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
  return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
  """Runs basic whitespace cleaning and splitting on a piece of text."""
  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens


class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True, split_on_punc=True):
    self.vocab = load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    self.basic_tokenizer = BasicTokenizer(
        do_lower_case=do_lower_case, split_on_punc=split_on_punc)
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)


class BasicTokenizer(object):
  """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

  def __init__(self, do_lower_case=True, split_on_punc=True):
    """Constructs a BasicTokenizer.

    Args:
      do_lower_case: Whether to lower case the input.
      split_on_punc: Whether to apply split on punctuations. By default BERT
        starts a new token for punctuations. This makes detokenization difficult
        for tasks like seq2seq decoding.
    """
    self.do_lower_case = do_lower_case
    self.split_on_punc = split_on_punc

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    text = self._clean_text(text)

    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    text = self._tokenize_chinese_chars(text)

    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
      if self.do_lower_case:
        token = token.lower()
        token = self._run_strip_accents(token)
      if self.split_on_punc:
        split_tokens.extend(self._run_split_on_punc(token))
      else:
        split_tokens.append(token)

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens

  def _run_strip_accents(self, text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
      cat = unicodedata.category(char)
      if cat == "Mn":
        continue
      output.append(char)
    return "".join(output)

  def _run_split_on_punc(self, text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ["".join(x) for x in output]

  def _tokenize_chinese_chars(self, text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
      cp = ord(char)
      if self._is_chinese_char(cp):
        output.append(" ")
        output.append(char)
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

  def _is_chinese_char(self, cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
      return True

    return False

  def _clean_text(self, text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
      cp = ord(char)
      if cp == 0 or cp == 0xfffd or _is_control(char):
        continue
      if _is_whitespace(char):
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)


class WordpieceTokenizer(object):
  """Runs WordPiece tokenziation."""

  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    """Tokenizes a piece of text into its word pieces.

    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.

    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]

    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.

    Returns:
      A list of wordpiece tokens.
    """

    text = convert_to_unicode(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        continue

      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          if start > 0:
            substr = "##" + substr
          if substr in self.vocab:
            cur_substr = substr
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end

      if is_bad:
        output_tokens.append(self.unk_token)
      else:
        output_tokens.extend(sub_tokens)
    return output_tokens


def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically control characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False


def preprocess_text(inputs, remove_space=True, lower=False):
  """Preprocesses data by removing extra space and normalize data.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py

  Args:
    inputs: The input text.
    remove_space: Whether to remove the extra space.
    lower: Whether to lowercase the text.

  Returns:
    The preprocessed text.

  """
  outputs = inputs
  if remove_space:
    outputs = " ".join(inputs.strip().split())

  if six.PY2 and isinstance(outputs, str):
    try:
      outputs = six.ensure_text(outputs, "utf-8")
    except UnicodeDecodeError:
      outputs = six.ensure_text(outputs, "latin-1")

  outputs = unicodedata.normalize("NFKD", outputs)
  outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, sample=False):
  """Segements text into pieces.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py


  Args:
    sp_model: A spm.SentencePieceProcessor object.
    text: The input text to be segemented.
    sample: Whether to randomly sample a segmentation output or return a
      deterministic one.

  Returns:
    A list of token pieces.
  """
  if six.PY2 and isinstance(text, six.text_type):
    text = six.ensure_binary(text, "utf-8")

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    piece = printable_text(piece)
    if len(piece) > 1 and piece[-1] == "," and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(piece[:-1].replace(
          SPIECE_UNDERLINE, ""))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  """Segments text and return token ids.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py

  Args:
    sp_model: A spm.SentencePieceProcessor object.
    text: The input text to be segemented.
    sample: Whether to randomly sample a segmentation output or return a
      deterministic one.

  Returns:
    A list of token ids.
  """
  pieces = encode_pieces(sp_model, text, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids


class FullSentencePieceTokenizer(object):
  """Runs end-to-end sentence piece tokenization.

  The interface of this class is intended to keep the same as above
  `FullTokenizer` class for easier usage.
  """

  def __init__(self, sp_model_file):
    """Inits FullSentencePieceTokenizer.

    Args:
      sp_model_file: The path to the sentence piece model file.
    """
    self.sp_model = spm.SentencePieceProcessor()
    self.sp_model.Load(sp_model_file)
    self.vocab = {
        self.sp_model.IdToPiece(i): i
        for i in six.moves.range(self.sp_model.GetPieceSize())
    }

  def tokenize(self, text):
    """Tokenizes text into pieces."""
    return encode_pieces(self.sp_model, text)

  def convert_tokens_to_ids(self, tokens):
    """Converts a list of tokens to a list of ids."""
    return [self.sp_model.PieceToId(printable_text(token)) for token in tokens]

  def convert_ids_to_tokens(self, ids):
    """Converts a list of ids ot a list of tokens."""
    return [self.sp_model.IdToPiece(id_) for id_ in ids]

In [17]:
# Create tokenizer " Instantiate FullTokenizer"
tokenizer = FullTokenizer(vocab_file, do_lower_case)
# name must be "tokenizer"

In [18]:
'''
Create train and test tokens (X_train_tokens, X_test_tokens) from (X_train, X_test) using Tokenizer and 
add '[CLS]' at start of the Tokens and '[SEP]' at the end of the tokens.
'''
def get_tokens_mask_segment(Text):
  # get the tokens
  tokens = tokenizer.tokenize(Text)
  
  # marking begining and end of the sentence by prepending and appending delimiters
  tokens = ["[CLS]", *tokens, "[SEP]"]
  
  # maximum number of tokens is 55(We already given this to BERT layer above) so shape is (None, 55)
  # if it is less than 55, add '[PAD]' token else truncate the tokens length.(similar to padding)
  # Based on padding, create the mask for Train and Test ( 1 for real token, 0 for '[PAD]'), 
  # it will also same shape as input tokens (None, 55) save those in X_train_mask, X_test_mask
  l = len(tokens)
  off_by = max_seq_length - len(tokens)
  if off_by >0:
    tokens = np.array(tokens + ["[PAD]"] * off_by)
    mask = np.array([1] * l + ([0] * off_by))
  else:
    tokens = np.array(tokens[:max_seq_length-1] + ["[SEP]"])
    mask = np.array([1] * max_seq_length)
  
  segment = np.array([0] * max_seq_length)

  return tokens, mask, segment

# -------------------------text column-------------
X_train_tokens_text, X_train_mask_text, X_train_segment_text = [[], [], []]

X_val_tokens_text, X_val_mask_text, X_val_segment_text = [[], [], []]

X_test_tokens_text, X_test_mask_text, X_test_segment_text = [[], [], []]

X_kaggle_tokens_text, X_kaggle_mask_text, X_kaggle_segment_text = [[], [], []]

# -------------------------parent text-----------

X_train_tokens_parent_text, X_train_mask_parent_text, X_train_segment_parent_text = [[], [], []]

X_val_tokens_parent_text, X_val_mask_parent_text, X_val_segment_parent_text = [[], [], []]

X_test_tokens_parent_text, X_test_mask_parent_text, X_test_segment_parent_text = [[], [], []]

X_kaggle_tokens_parent_text, X_kaggle_mask_parent_text, X_kaggle_segment_parent_text = [[], [], []]

# ------------------text column------------------

for text in X_train.text.values:
  tokens, mask, segment = get_tokens_mask_segment(text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_train_tokens_text.append(tokens)
  X_train_mask_text.append(mask)
  X_train_segment_text.append(segment)

for text in X_val.text.values:
  tokens, mask, segment = get_tokens_mask_segment(text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_val_tokens_text.append(tokens)
  X_val_mask_text.append(mask)
  X_val_segment_text.append(segment)

for text in X_test.text.values:
  tokens, mask, segment = get_tokens_mask_segment(text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_test_tokens_text.append(tokens)
  X_test_mask_text.append(mask)
  X_test_segment_text.append(segment)

# ------------------parent text column------------------
for parent_text in X_train.parent_text.values:
  tokens, mask, segment = get_tokens_mask_segment(parent_text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_train_tokens_parent_text.append(tokens)
  X_train_mask_parent_text.append(mask)
  X_train_segment_parent_text.append(segment)

for parent_text in X_val.parent_text.values:
  tokens, mask, segment = get_tokens_mask_segment(parent_text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_val_tokens_parent_text.append(tokens)
  X_val_mask_parent_text.append(mask)
  X_val_segment_parent_text.append(segment)

for parent_text in X_test.parent_text.values:
  tokens, mask, segment = get_tokens_mask_segment(parent_text)
  tokens = tokenizer.convert_tokens_to_ids(tokens)
  X_test_tokens_parent_text.append(tokens)
  X_test_mask_parent_text.append(mask)
  X_test_segment_parent_text.append(segment)

# type cast to numpy array

# --------------------text column---------------
X_train_tokens_text = np.array(X_train_tokens_text)
X_train_mask_text = np.array(X_train_mask_text)
X_train_segment_text = np.array(X_train_segment_text)


X_val_tokens_text = np.array(X_val_tokens_text)
X_val_mask_text = np.array(X_val_mask_text)
X_val_segment_text = np.array(X_val_segment_text)

X_test_tokens_text = np.array(X_test_tokens_text)
X_test_mask_text = np.array(X_test_mask_text)
X_test_segment_text = np.array(X_test_segment_text)

#------------------------parent text--------------
X_train_tokens_parent_text = np.array(X_train_tokens_parent_text)
X_train_mask_parent_text = np.array(X_train_mask_parent_text)
X_train_segment_parent_text = np.array(X_train_segment_parent_text)


X_val_tokens_parent_text = np.array(X_val_tokens_parent_text)
X_val_mask_parent_text = np.array(X_val_mask_parent_text)
X_val_segment_parent_text = np.array(X_val_segment_parent_text)

X_test_tokens_parent_text = np.array(X_test_tokens_parent_text)
X_test_mask_parent_text = np.array(X_test_mask_parent_text)
X_test_segment_parent_text = np.array(X_test_segment_parent_text)

# Getting Embeddings from BERT Model

In [19]:
# -----------------------text column------------------
# get the train, validation and test embeddings for "text" column
X_train_pooled_output_text = bert_model.predict([X_train_tokens_text, X_train_mask_text, X_train_segment_text], batch_size= 64)
X_val_pooled_output_text = bert_model.predict([X_val_tokens_text, X_val_mask_text, X_val_segment_text], batch_size= 64)
X_test_pooled_output_text = bert_model.predict([X_test_tokens_text, X_test_mask_text, X_test_segment_text], batch_size= 64)

# -----------------------parent text --------------------
# get the train, validation and test embeddings for "parent text" column
X_train_pooled_output_parent_text = bert_model.predict([X_train_tokens_parent_text, X_train_mask_parent_text, X_train_segment_parent_text], batch_size= 64)
X_val_pooled_output_parent_text = bert_model.predict([X_val_tokens_parent_text, X_val_mask_parent_text, X_val_segment_parent_text], batch_size= 64)
X_test_pooled_output_parent_text = bert_model.predict([X_test_tokens_parent_text, X_test_mask_parent_text, X_test_segment_parent_text], batch_size= 64)

16/16 [==============================] - 3829s 235s/step


**Note**

Expected Time taken to run above cell is 7 Hours 54 min (474m 21.5s)

# Saving embedding to the disk

In [20]:
pickle.dump((X_train_pooled_output_text, X_val_pooled_output_text, X_test_pooled_output_text),open(('embeddings_text.pkl'),'wb'))
pickle.dump((X_train_pooled_output_parent_text, X_val_pooled_output_parent_text, X_test_pooled_output_parent_text),open(('embeddings_parent_text.pkl'),'wb'))

# Load saved embeddings

In [21]:
# X_train_pooled_output,X_val_pooled_output, X_test_pooled_output = pickle.load(open(('embeddings_text.pkl'), 'rb'))
# X_train_pooled_output_parent_text, X_val_pooled_output_parent_text, X_test_pooled_output_parent_text = pickle.load(open(('embeddings_parent_text.pkl'), 'rb'))

# Append remaining features
We will append embeddings + numeric features

In [22]:
def add_text_length(df, compi= 1):
    """
    add word count feature
    """
    # adding length column
    df["text_len"] = df.text.apply(lambda x: len(x.split()))
    df["parent_text_len"] = df.parent_text.apply(lambda x: len(x.split()))
    return df

X_train = add_text_length(X_train.copy())
X_val = add_text_length(X_val.copy())
X_test = add_text_length(X_test.copy())

In [23]:
# append remaining features
num_train = X_train[['controversiality', 'parent_score',
       'parent_controversiality', 'text_len', 'parent_text_len']]

num_val = X_val[['controversiality', 'parent_score',
       'parent_controversiality', 'text_len', 'parent_text_len']]

num_test = X_test[['controversiality', 'parent_score',
       'parent_controversiality', 'text_len', 'parent_text_len']]

# sentiment feature list
sentiment_score_list = ["neg_text",
                        "neu_text",
                        "pos_text",
                        "compound_text",
                        "neg_parent_text",
                        "neu_parent_text",
                        "pos_parent_text",
                        "compound_parent_text"]
train_sentiment = X_train[sentiment_score_list].values
test_sentiment = X_test[sentiment_score_list].values
val_sentiment = X_val[sentiment_score_list].values

In [24]:
X_train_final = np.concatenate([X_train_pooled_output_text, X_train_pooled_output_parent_text, num_train, train_sentiment], axis= 1)
X_val_final = np.concatenate([X_val_pooled_output_text, X_val_pooled_output_parent_text, num_val, val_sentiment], axis= 1)
X_test_final = np.concatenate([X_test_pooled_output_text, X_test_pooled_output_parent_text, num_test, test_sentiment], axis= 1)

y_train = X_train.Score
y_val = X_val.Score

In [25]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train_final.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(16, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(8, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))
op = Adam(lr= 0.0001)
# Compile the network :
NN_model.compile(loss='MSE', optimizer=op)
NN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               198400    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 209409 (818.00 KB)
Trainable params: 20940

In [26]:
# callbacks and checkpoints
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint_path = "best.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
callback_list = [
ModelCheckpoint(checkpoint_path, monitor = 'val_loss',verbose = 1,save_weights_only=True, save_best_only = True,mode="min"),
EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min'),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min',min_delta=1e-4)
]

In [27]:
# fit the data
NN_model.fit(X_train_final, 
             y_train, 
             validation_data=(X_val_final,  y_val), 
             epochs=500, 
             batch_size=32, 
             callbacks=callback_list)

Epoch 1/500



141/141 [==============================] - ETA: 0s - loss: 38148.7266
Epoch 1: val_loss improved from inf to 22899.44727, saving model to best.ckpt
141/141 [==============================] - 90s 329ms/step - loss: 38148.7266 - val_loss: 22899.4473 - lr: 0.0010
Epoch 2/500
140/141 [============================>.] - ETA: 0s - loss: 29280.2227
Epoch 2: val_loss improved from 22899.44727 to 21117.12695, saving model to best.ckpt
141/141 [==============================] - 38s 271ms/step - loss: 29209.1367 - val_loss: 21117.1270 - lr: 0.0010
Epoch 3/500
140/141 [============================>.] - ETA: 0s - loss: 29085.8555
Epoch 3: val_loss improved from 21117.12695 to 20905.56836, saving model to best.ckpt
141/141 [==============================] - 108s 772ms/step - loss: 29249.1973 - val_loss: 20905.5684 - lr: 0.0010
Epoch 4/500
141/141 [==============================] - ETA: 0s - loss: 28424.1289
Epoch 4: val_loss improved from 20905.56836 to 19513.65820, saving model to best.ckpt
141/141 

# Making predictions

In [28]:
# predictions
pred = NN_model.predict(X_test_final)
# to dataframe
df_submit = pd.DataFrame({"prediction": pred.flatten()})
# save to the disk
df_submit.to_csv("submission.csv", index= False)

32/32 [==============================] - 20s 9ms/step


**The submission of this baseline model yields 190.6713 RMSE on public leaderboard**